I did not find the gold labels for the test set, so the only way to check your performance on the test set is to make a submission on the [Kaggle website](https://www.kaggle.com/c/nlp-getting-started/submit). 
I made a function to create a submission file from a prediction vector, *download_predictions_to_file()*, feel free to use it. Alternatively, you can just check your performance on a subset of the training set. 

In [1]:
# some libraries
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
#from sklearn.feature_extraction.text import CountVectorizer 
#from sklearn.naive_bayes import MultinomialNB

In [2]:
import numpy as np
import pandas as pd 

data_url = 'https://www.math.unipd.it/~dasan/disaster/'
train_csv = pd.read_csv(data_url + 'train.csv', sep=",") 
test_csv = pd.read_csv(data_url + 'test.csv', sep=",") 

In [3]:
def download_predictions_to_file(id_array, predictions_array):
  # This function, given a numpy array with the ids of the test tweets and a numpy array with the corresponding predictions, 
  # creates and let's you download a prediction file suitable to be uploaded to the Kaggle website

  id_array = id_array.reshape(len(id_array),1) # making sure the arrays are of the size requested by hstack(), i.e. (x,1) instead of (x,) 
  predictions_array = predictions_array.reshape(len(predictions_array),1)

  from google.colab import files
  result = pd.DataFrame(np.hstack((id_array, predictions_array)), columns=['id','target'])
  result.to_csv('prediction_file.csv', index=False)
  files.download('prediction_file.csv')

In [4]:
# let's take a look at the data
print(train_csv.head())

Y_train = train_csv["target"].values 
X_train = train_csv.values[:, 1:4] # extracting the columns keyword, location and text

   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  


In [5]:
# Get only column with tweet text
x_tweets = train_csv['text']
x_tweets.head()

0    Our Deeds are the Reason of this #earthquake M...
1               Forest fire near La Ronge Sask. Canada
2    All residents asked to 'shelter in place' are ...
3    13,000 people receive #wildfires evacuation or...
4    Just got sent this photo from Ruby #Alaska as ...
Name: text, dtype: object

In [6]:
# Tweet tokenizer for making all alphabetical symbols lowercase, reducing length of words
# if they are e.g. 'goaaaal' and for removing account handles
tknzr = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
tweet_tokens = []
for sent in x_tweets:
    #print(tweet_tokenizer.tokenize(sent))
    tweet_tokens.append(tknzr.tokenize(sent))
print(tweet_tokens[:2])

[['our', 'deeds', 'are', 'the', 'reason', 'of', 'this', '#earthquake', 'may', 'allah', 'forgive', 'us', 'all'], ['forest', 'fire', 'near', 'la', 'ronge', 'sask', '.', 'canada']]


In [7]:
import nltk
import string
import re
from nltk.corpus import stopwords
english_stopwords = stopwords.words("english")
# print(english_stopwords)

In [8]:
# clean out stopwords, # signs, punctuation and numbers
clean_t2 = [[word if (word.startswith('#') == False) else re.sub(r'#', '', word) for word in tweet] for tweet in tweet_tokens]
clean_t3 = [[word for word in tweet if (word not in string.punctuation) and (word.isalpha() == True) and (word not in english_stopwords)] for tweet in clean_t2]

In [9]:
clean_t3[:2]

[['deeds', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'us'],
 ['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada']]

In [10]:
# Turns out this step is necessary to have a correct input form for the Vectorizer
clean_t4 = [' '.join(tweet) for tweet in clean_t3]
x_tweets = np.array(clean_t4)
x_tweets[:2]

array(['deeds reason earthquake may allah forgive us',
       'forest fire near la ronge sask canada'], dtype='<U138')

In [11]:
# Perform the split into training and test set with 'x_tweets' as input
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_tweets, Y_train, test_size=0.15, random_state=42)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer
tf_computer = TfidfVectorizer(use_idf=False, ngram_range=(1,1), stop_words='english')

train_features = tf_computer.fit_transform(X_train)
test_features = tf_computer.transform(X_test)

print("vocabulary size:", len(tf_computer.vocabulary_))

vocabulary size: 12272


Now we have the proper input format to start testing models

### Unused code

In [ ]:
##### for tokenizing the keywords ##### MIGHT BE USEFUL IF WE CHOOSE KEYWORD AS A FEATURE
# for i, value in enumerate(x_feat[0][:]):
#    if type(value) != float:
#        x_feat[0][i]=tknzr_3.tokenize(value)

### Check balance

In [ ]:
# is the number of disaster vs not disaster tweets balanced? 
print('disaster tweets = {}, not disaster = {}'.format(sum(Y_train==1), sum(Y_train == 0)))